# Imágenes tomadas con una cámara

Una vez comprobado que el programa detecta bien los defectos de nuestras imágenes de prueba, lo comprobamos con frutas a las que les hemos tomado una fotografía para comprobar que no funciona solo con imágenes en un entorno perfecto, y se podría hacer en cualquier lugar.

Este cuaderno es implementar el mismo código que nos encontramos en la Parte-2, pero con otra imagen, por lo que no variará la explicación de cada apartado.

En primer lugar cargamos la imagen para ver al inicio como es, así como realizar un preprocesamiento de la imagen, donde aplicamos un suavizado por mediana, para eliminar todo tipo de ruido que pueda haber, manteniendo los bordes intactos para poder analizarlos bien más adelante

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Función para mostrar imágenes
def show(img, title="", cmap=None):
    plt.figure(figsize=(6,6))
    if cmap is None:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Cargar imagen
path = './images/'
img = cv2.imread(path + 'Platano1.jpg')
original = img.copy()


# Comprobamos si la imagen se cargó correctamente
if img is None:
    print("Error: No se pudo cargar la imagen. Revisa la ruta.")
else:
    show(original, "Imagen Original")


# Suavizado para eliminar ruido
blurred = cv2.medianBlur(img, 5)

Deberemos de aplicar un histograma para detectar las zonas más oscuras de la imagen. En lugar de considerar oscuro todo lo que esté por debajo de un valor fijo, usaremos el algoritmo de Otsu. Este analiza el histograma de luz de la foto y nos calcula matemáticamente el punto exacto que nos separa la piel clara de las manchas más oscuras.

Esto nos generará una máscara en blanco y negro, donde lo blanco son los posibles defectos oscuros.

In [ ]:
# Detección de zonas oscuras (Histograma)
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
hist = cv2.calcHist([gray], [0], None, [256], [0,256])

# Buscamos el pico de oscuridad en los primeros 100 valores y sumamos un margen
shadow_thresh = np.argmax(hist[:100]) + 5
_, shadow_mask = cv2.threshold(gray, shadow_thresh, 255, cv2.THRESH_BINARY)

# Limpiamos ruido en la máscara y la invertimos para que lo blanco sea lo oscuro
shadow_mask = cv2.medianBlur(shadow_mask, 9)
shadow_mask = cv2.bitwise_not(shadow_mask)

show(shadow_mask, "Máscara de Zonas Oscuras", cmap='gray')

En la imagen podemos ver que detecta de forma bastante buena el área de la imagen que se encuentra defectuosa.

Una vez obtenido el histograma, buscaremos si puede llegar a haber algún falso positivo, ya que el paso anterior nos puede detectar zonas que no son defectuosas, como el tallo de la manzana o una sombra algo pequeña.

Para comprobar que todo ha salido bien, el código buscará el defecto de dos formas a la vez: 

- Por el color, que busca los píxeles que tengan un color algo raro.

In [ ]:
# Detección por rango de color (HSV)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

lower_defect = np.array([0, 30, 0])
upper_defect = np.array([180, 255, 120])

color_mask = cv2.inRange(hsv, lower_defect, upper_defect)

show(color_mask, "Candidatos por Color/Brillo", cmap='gray')

- Por bordes, usando el algoritmo de Canny, que busca los cambios bruscos en la superficie.

In [ ]:
# Detección de bordes
edges = cv2.Canny(gray, 40, 120)

# Hacemos grandes los bordes para conectarlos mejor
edges = cv2.dilate(edges, np.ones((5,5), np.uint8), iterations=2)

show(edges, "Bordes Detectados", cmap='gray')

Finalmente, nuestro código tomará todas los pasos que hemos realizado previamente para dibujar el contorno final.

Se marcará el área de defecto si cumple ambas condiciones:
- Tiene un borde extraño o tiene un color extraño.
- Está dentro de la zona oscura detectada por el algoritmo de Otsu.

Esto nos detectará si hay una sombra pero no tiene bordes de textura, el código la ignorará, y si tien bordes pero no es oscura, el código la ignorará.

In [ ]:
# Solo aceptamos bordes que caigan dentro de la zona oscura
defect_edges = cv2.bitwise_and(edges, edges, mask=shadow_mask)

# Solo aceptamos defectos de color que caigan dentro de la zona oscura
defect_color = cv2.bitwise_and(color_mask, color_mask, mask=shadow_mask)

# Combinación
combined = cv2.bitwise_or(defect_edges, defect_color)

# Si la combinación borra todo, volvemos al color)
if np.sum(combined) < 20:
    print("Aviso: Detección combinada muy baja, usando solo color.")
    combined = defect_color

# Limpieza final de la máscara combinada
combined = cv2.medianBlur(combined, 7)
show(combined, "Máscara Combinada FINAL", cmap='gray')

In [ ]:
# Mostramos resultado final con contornos
resultado = original.copy()
contornos, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dibujamos en azul los contornos detectados
cv2.drawContours(resultado, contornos, -1, (255, 0, 0), thickness=3)

show(resultado, "Resultado Final")

A continuación, implementaremos otra imagen para comprobar que funciona sin ningún fallo. Mostraremos solo la imagen original y la imagen con la zona marcada para evitar revisar todas las máscaras.

In [ ]:
# Cargar imagen
path = './images/'
img = cv2.imread(path + 'Platano2.jpg')
original = img.copy()


# Comprobamos si la imagen se cargó correctamente
if img is None:
    print("Error: No se pudo cargar la imagen. Revisa la ruta.")
else:
    show(original, "Imagen Original")


# Suavizado para eliminar ruido
blurred = cv2.medianBlur(img, 5)


# Detección de zonas oscuras (Histograma)
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
hist = cv2.calcHist([gray], [0], None, [256], [0,256])

# Buscamos el pico de oscuridad en los primeros 100 valores y sumamos un margen
shadow_thresh = np.argmax(hist[:100]) + 5
_, shadow_mask = cv2.threshold(gray, shadow_thresh, 255, cv2.THRESH_BINARY)

# Limpiamos ruido en la máscara y la invertimos para que lo blanco sea lo oscuro
shadow_mask = cv2.medianBlur(shadow_mask, 9)
shadow_mask = cv2.bitwise_not(shadow_mask)


# Detección por rango de color (HSV)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

lower_defect = np.array([0, 30, 0])
upper_defect = np.array([180, 255, 120])

color_mask = cv2.inRange(hsv, lower_defect, upper_defect)


# Detección de bordes
edges = cv2.Canny(gray, 40, 120)

# Hacemos grandes los bordes para conectarlos mejor
edges = cv2.dilate(edges, np.ones((5,5), np.uint8), iterations=2)


# Solo aceptamos bordes que caigan dentro de la zona oscura
defect_edges = cv2.bitwise_and(edges, edges, mask=shadow_mask)

# Solo aceptamos defectos de color que caigan dentro de la zona oscura
defect_color = cv2.bitwise_and(color_mask, color_mask, mask=shadow_mask)

# Combinación
combined = cv2.bitwise_or(defect_edges, defect_color)

# Si la combinación borra todo, volvemos al color)
if np.sum(combined) < 20:
    print("Aviso: Detección combinada muy baja, usando solo color.")
    combined = defect_color

# Limpieza final de la máscara combinada
combined = cv2.medianBlur(combined, 7)

# Mostramos resultado final con contornos
resultado = original.copy()
contornos, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dibujamos en azul los contornos detectados
cv2.drawContours(resultado, contornos, -1, (255, 0, 0), thickness=3)

show(resultado, "Resultado Final")

### Pruebas de imágenes con fondos distintos

In [ ]:
# Cargamos el modelo YOLOv8 preentrenado para segmentación
model = YOLO('yolov8n-seg.pt')

path = './images/'
filename = 'P2.jpg' 

img = cv2.imread(path + filename)

if img is None:
    print(f"Error: No se encuentra {filename} en la ruta {path}")
else:
    original = img.copy()
    show(original, f"Imagen Original")





results = model(img, conf=0.4, verbose=False)

# Creamos una máscara vacía (todo negro) por defecto
mask_fruta = np.zeros(img.shape[:2], dtype=np.uint8)
max_area = 0

# Buscamos la fruta más grande
if results[0].masks is not None:
    masks = results[0].masks.data.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    names = results[0].names
    
    for i, class_id in enumerate(classes):
        name = names[int(class_id)]
        
        # Lista de cosas que consideramos "Fruta"
        if name in ['banana', 'apple', 'pear', 'orange']:
            # Redimensionar máscara
            mask_raw = masks[i]
            mask_resized = cv2.resize(mask_raw, (img.shape[1], img.shape[0]))
            current_mask = (mask_resized * 255).astype(np.uint8)
            
            # Calculamos área
            area = cv2.countNonZero(current_mask)
            
            # LEY DEL MÁS FUERTE: Nos quedamos solo con la fruta más grande
            if area > max_area:
                max_area = area
                mask_fruta = current_mask





# Suavizado para eliminar ruido
blurred = cv2.medianBlur(img, 5)

# Detección de zonas oscuras (Histograma)
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

# solo calcule los colores DE LA FRUTA, no del fondo.
hist = cv2.calcHist([gray], [0], mask_fruta, [256], [0,256])

# Buscamos el pico de oscuridad en los primeros 100 valores y sumamos un margen
shadow_thresh = np.argmax(hist[:100]) + 5
_, shadow_mask = cv2.threshold(gray, shadow_thresh, 255, cv2.THRESH_BINARY)

# Limpiamos ruido en la máscara y la invertimos para que lo blanco sea lo oscuro
shadow_mask = cv2.medianBlur(shadow_mask, 9)
shadow_mask = cv2.bitwise_not(shadow_mask)

# CAMBIO PEQUEÑO 2: Recortamos la sombra para que no detecte el fondo negro como "defecto"
shadow_mask = cv2.bitwise_and(shadow_mask, shadow_mask, mask=mask_fruta)


# Detección por rango de color (HSV)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

lower_defect = np.array([0, 30, 0])
upper_defect = np.array([180, 255, 120])

color_mask = cv2.inRange(hsv, lower_defect, upper_defect)

# Aseguramos que el color defectuoso esté dentro de la fruta
color_mask = cv2.bitwise_and(color_mask, color_mask, mask=mask_fruta)


# Detección de bordes
edges = cv2.Canny(gray, 40, 120)

# Hacemos grandes los bordes para conectarlos mejor
edges = cv2.dilate(edges, np.ones((5,5), np.uint8), iterations=2)

# Quitamos los bordes del fondo
edges = cv2.bitwise_and(edges, edges, mask=mask_fruta)


# Solo aceptamos bordes que caigan dentro de la zona oscura
defect_edges = cv2.bitwise_and(edges, edges, mask=shadow_mask)

# Solo aceptamos defectos de color que caigan dentro de la zona oscura
defect_color = cv2.bitwise_and(color_mask, color_mask, mask=shadow_mask)

# Combinación
combined = cv2.bitwise_or(defect_edges, defect_color)

# Si la combinación borra todo, volvemos al color
if np.sum(combined) < 20:
    print("Aviso: Detección combinada muy baja, usando solo color.")
    combined = defect_color

# Limpieza final de la máscara combinada
combined = cv2.medianBlur(combined, 7)

# ULTIMO FILTRO DE SEGURIDAD:
combined = cv2.bitwise_and(combined, combined, mask=mask_fruta)

# Mostramos resultado final con contornos
resultado = original.copy()
contornos, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dibujamos en azul los contornos detectados
cv2.drawContours(resultado, contornos, -1, (255, 0, 0), thickness=3)

show(resultado, "Resultado Final")